<a href="https://colab.research.google.com/github/andreistepanyuk/STP_prediction/blob/main/Regression_genes_to_STP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Regression genes->STP model: train and test**

Notebook for comparison of different models of genes->stp dependences with respect to generalization (including hierarhical models taking into account structure of transcriptomic space). test MDL based regularization approaches

**Test GPU availability**

In [1]:
import torch
print('Number of gpus available: ',torch.cuda.device_count())
print('Name of gpu[0]: ',torch.cuda.get_device_name(0))

Number of gpus available:  1
Name of gpu[0]:  Tesla K80


**Mount Google Drive**

In [2]:
# Mount Google Drive

from google.colab import drive

drive.mount('/content/drive')

#! pip3 install numpy==1.15.4 

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load packages

In [3]:
!pip3 install pandas pymc3 ipython  jupyter jinja2  scikit-learn  # theano

Upgrade tables package

In [4]:
! pip3 install --upgrade tables

Install gpytorch 

In [5]:
!pip3 install gpytorch

In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [7]:
import sklearn as sk
from sklearn import linear_model
# Test ridge regression
reg = sk.linear_model.Ridge(alpha=.5)
reg.fit([[0, 0], [0, 0], [1, 1]], [0, .1, 1])

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

**Import module for genes->stp modeling**

In [8]:
#'/content/drive/My Drive/Colab Notebooks/'
import genes2stp as g2p

IndentationError: ignored

**Select GPU device**

In [ ]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
cuda0 = torch.device('cuda:0')

1
Tesla P100-PCIE-16GB


**Load gene set filter**

In [ ]:
import os
print(os.getcwd())

d5 = '/content/drive/My Drive/Colab Notebooks/'
df_ge_names_filter = pd.read_excel(d5+'gene_set_names.xlsx',header=None).loc[1:,1]
df_ge_names_filter

/content


1        pre__Slit1
2        pre__Slit2
3        pre__Slit3
4        pre__Robo1
5        pre__Robo2
           ...     
1781    post__Galr1
1782    post__Galr2
1783    post__Galr3
1784    post__Elfn1
1785    post__Elfn2
Name: 1, Length: 1785, dtype: object

**Train and test regression model**

In [ ]:
gs_name = '_gs5188' #'_gs1512' #'_gs15656' #'_gs1512' #'_gs15656' #'_gs1512' #'_gs15656' #'_gs1512'   #'_gs5188' #'_gs1512'
d4 = '/content/drive/My Drive/Colab Notebooks/scVI/'

fname_columns = d4+'stp_to_ge_all_columns'+gs_name+'.hdf'
fname = d4+'stp_to_ge_all_data'+gs_name+'.hdf'

X_train0, ge_columns_train, annot_columns_train, stp_columns_train, classes_columns_train =
g2p.load_genes_and_stp(fname, fname_columns=fname_columns)

print('X_train0 shape',X_train0.shape)
print(X_train0.columns[200:300])
print(X_train0.columns[300:])


print('gs_name - '+gs_name)

# add scVI latent factors names to ge_columns_train
if (gs_name=='_gs5188')|(gs_name=='_gs1512')|(gs_name=='_gs15656'):
    nlf = 20   # 20 were in scvi model
elif (gs_name=='_gs219'):
    nlf = 10   # 10 were in scvi model
lf_names = np.arange(nlf).astype(str)
lf_names1 =  ['pre__'+s for s in lf_names.tolist()]
lf_names2 =  ['post__'+s for s in lf_names.tolist()]

lf_scvi_names = lf_names1 + lf_names2

ge_columns_train2 = list(set(ge_columns_train).difference(set(lf_scvi_names)))
lf_scvi_names


#Add several classification columns
#### modify data: DO class_pre, class_post, subclass_pre, subclass_post columns
subcl_names_pre = ['ex_ctx__pre', 'ex_ec__pre','ex_hipp__pre','sst__pre','pvalb__pre','cge__pre']
subcl_names_post = ['ex_ctx__post', 'ex_ec__post','ex_hipp__post','sst__post','pvalb__post','cge__post']
subcl_names_pre_b = ['ex_ctx__pre', 'ex_ec__pre','ex_hipp__pre','inh__pre']
subcl_names_pre_d = ['ex_ctx__pre', 'ex_ec__pre','ex_hipp__pre','sst__pre','pvalb__pre','vip__pre','cck__pre','lamp5__pre']
subcl_names_post_c = ['ex__post','sst__post','pvalb__post','cge__post']
subcl_names_pre_c = ['ex__pre','sst__pre','pvalb__pre','cge__pre']
subcl_names_post_b = ['ex_ctx__post', 'ex_ec__post','ex_hipp__post','inh__post']
new_syntype_columns = pd.DataFrame([['class_pre',['ex__pre', 'inh__pre'], ['Glutamatergic', 'GABAergic']],
                                    ['subclass_pre',subcl_names_pre, subcl_names_pre],
                                    ['subclass_pre_b',subcl_names_pre_b, subcl_names_pre_b],
                                    ['subclass_pre_d',subcl_names_pre_d, subcl_names_pre_d],
                                    ['subclass_post_c',subcl_names_post_c, subcl_names_post_c],
                                    ['subclass_pre_c',subcl_names_pre_c, subcl_names_pre_c],
                                    ['subclass_post_b',subcl_names_post_b, subcl_names_post_b],
                                    ['class_post',['ex__post', 'inh__post'], ['Glutamatergic', 'GABAergic']],
                                    ['subclass_post',subcl_names_post, subcl_names_post]], columns = ['group','columns_X0','new_names'])

X_train0, classes_columns_train = g2p.add_class_hierarchy(new_syntype_columns,X_train0,classes_columns_train)

**preprocess genes_stp data**

In [ ]:
X2, y2, X2_cl, X2_an, sts, preprocessing_, cla_n2 =
g2p.data_preprocessing(X_train0, N_bootstraps=100, Dn=1, stp_n=['A5_20Hz'], classes_columns_train=classes_columns_train, 
                       df_ge_names_filter=df_ge_names_filter,lf_scvi_names=lf_scvi_names,
                       remove_st=[4,90],d_log=0.3):

**train and test genes->STP model**

In [ ]:
# make models structure
parameters = {'sig' : 0.4, 'lyambda0' : 50, 'alpha_sigma_factor':0.01, 'beta_sigma_factor':2.0,
              'alpha_lambda_factor':0.0000001, 'beta_lambda_factor':1.0,'nit':5, 'it_stop':0.0000001, 'n_jumps':0, 'dn_jumps':0}
H_Models = pd.DataFrame([       ['HLM',
                                    [['class_pre', 'class_post'],
                                     ['subclass_pre_b', 'subclass_post_c'],
                                     ['subclass_pre', 'subclass_post']]],
                                    parameters],
                                    columns = ['name','structure','parameters']) 
H_Models

In [ ]:
Y_pred, Y_pred0, Samples_test,  R3 =
g2p.train_and_test_regression_models(X2, y2, X2_cl, X2_an, H_Models, preprocessing_, ncv = 10, sts=sts, cla_n2=cla_n2, cuda0=cuda0)

In [ ]:
X2, y2, X2_cl, X2_an, sts, preprocessing_, cla_n2 =
g2p.data_preprocessing(X_train0, N_bootstraps=100, Dn=1, stp_n=['A5_20Hz'], classes_columns_train=classes_columns_train, 
                       df_ge_names_filter=df_ge_names_filter,lf_scvi_names=lf_scvi_names,
                       remove_st=[4,90],d_log=0.3)

In [ ]:
H_Models = pd.DataFrame([       ['Msb1sc2',['subclass_pre_b', 'subclass_post_c'],{}],
                                    ],
                                    columns = ['name','structure','parameters'])
H_Models

In [ ]:
Y_pred, Y_pred0, Samples_test,  R3 =
g2p.train_and_test_regression_models_(X2, y2, X2_cl, X2_an, H_Models, preprocessing_, ncv = 10,do_log_y = 0, sts=sts, cla_n2=cla_n2, cuda0=cuda0)